First, let's roll our own state monad. We'll treat assignments just as sequences of individuals.

In [1]:
{-# LANGUAGE TupleSections #-}

import Data.Bifunctor (first)
import Control.Applicative (liftA2)
import Control.Monad (liftM,liftM2)

type E = Int

type G = [E]

dom :: [E]
dom = [1..10]

newtype S a = S { runS :: G -> (a,G)}

instance Functor S where
    fmap f m = S $ \g -> first f (runS m g)
    
instance Applicative S where
    pure a = S (a,)
    m <*> n = S $ \g ->
        let (f,g') = runS m g
            (x,g'') = runS n g'
        in (f x,g'')
        
newtype SS a = SS { runSS :: G -> [(a,G)]}

instance Functor SS where
    fmap f m = SS $ \g -> first f <$> runSS m g
    
instance Applicative SS where
    pure a = SS $ (:[]) <$> (a,)
    m <*> n = SS $ \g -> [(f x,g'')|(f,g') <- runSS m g, (x,g'') <- runSS n g']

instance Monad SS where
    return = pure
    m >>= k = SS $ \g ->
        let pairs = runSS m g
        in concat [(runSS $ k x) g' | (x,g') <- pairs ]

join :: SS (SS a) -> SS a
join m = m >>= id
 
intro :: S E -> S E
intro m = S $ \g ->
    let (x,g') = runS m g
    in (x,x:g')

pro :: S E
pro = S $ \(x:g) -> (x,g)

Line 29: Use fmap
Found:
\ g -> first f <$> runSS m g
Why not:
(fmap (first f) . runSS m)

In [2]:
test1 = ($ []) $ runS $ liftA2 (&&) (pure odd <*> (intro . pure $ 1)) (pure even <*> intro pro)
test1

Line 1: Use <$>
Found:
pure odd <*> (intro . pure $ 1)
Why not:
odd <$> (intro . pure $ 1)Line 1: Use <$>
Found:
pure even <*> intro pro
Why not:
even <$> intro pro

(False,[1])

In [3]:
test2 = ($ []) $ runS $ liftA2 (&&) (pure odd <*> (intro pro)) (pure even <*> intro (pure 1))
test2

Line 1: Use <$>
Found:
pure odd <*> (intro pro)
Why not:
odd <$> intro proLine 1: Redundant bracket
Found:
pure odd <*> (intro pro)
Why not:
pure odd <*> intro proLine 1: Use <$>
Found:
pure even <*> intro (pure 1)
Why not:
even <$> intro (pure 1)

: 

Let's roll our own continuation monad:

In [4]:
newtype K b a = K { (>>-) :: (a -> b) -> b}

instance Functor (K b) where
    fmap f m = K $ \k -> m >>- (k . f)
    
instance Applicative (K b) where
    pure x = K $ \k -> k x
    m <*> n = K $ \k -> m >>- \f -> n >>- \x -> k (f x)
    
liftS :: S a -> SS a
liftS m = SS $ \g -> [runS m g]

lower :: K (SS a) (S a) -> SS a
lower m = (>>-) m liftS

In [5]:
anEvenInt :: SS E
anEvenInt = SS $ \g -> [(n,g)|n <- dom, even n]

Let's see how maximally polymorphic `liftS` and `lower` allow a violation of crossover.

"One is less than its successor"

In [6]:
($ []) $ runS $ (liftA2 (<)) (intro $ pure 1) ((pure succ) <*> pro)

Line 1: Redundant bracket
Found:
(liftA2 (<)) (intro $ pure 1)
Why not:
liftA2 (<) (intro $ pure 1)Line 1: Redundant bracket
Found:
(pure succ) <*> pro
Why not:
pure succ <*> pro

(True,[])

In [7]:
($ []) $ runS $ (liftA2 (<)) ((pure succ) <*> pro) (intro $ pure 1)

Line 1: Redundant bracket
Found:
(liftA2 (<)) ((pure succ) <*> pro)
Why not:
liftA2 (<) ((pure succ) <*> pro)Line 1: Redundant bracket
Found:
(pure succ) <*> pro
Why not:
pure succ <*> pro

: 

In [8]:
obj = pure pure <*> (K $ (>>=) $ (liftS (intro $ pure 1)))

Line 1: Use <$>
Found:
pure pure <*> (K $ (>>=) $ (liftS (intro $ pure 1)))
Why not:
pure <$> (K $ (>>=) $ (liftS (intro $ pure 1)))Line 1: Move brackets to avoid $
Found:
pure pure <*> (K $ (>>=) $ (liftS (intro $ pure 1)))
Why not:
pure pure <*> K ((>>=) $ (liftS (intro $ pure 1)))Line 1: Redundant bracket
Found:
(>>=) $ (liftS (intro $ pure 1))
Why not:
(>>=) $ liftS (intro $ pure 1)Line 1: Redundant $
Found:
$
Why not:

"A successor to it is greater than 1"

In [9]:
($ []) $ runSS $ lower $ (liftA2 $ liftA2 (>)) (pure $ ((pure succ) <*> pro)) obj

Line 1: Redundant $
Found:
$
Why not:
Line 1: Redundant bracket
Found:
(pure succ) <*> pro
Why not:
pure succ <*> pro

[(True,[])]

The fix is to rigidly type `liftS`, and concomitantly `lower`.

# Dynamic closure

Dynamic closure is defined below. Note that it destroys discourse referents.

In [10]:
clo :: SS Bool -> S Bool
clo m = S $ \g -> (any fst (runSS m g),g)

Externally dynamic negation is just fmapped negation.

In [11]:
($ []) $ runS $ (liftA2 (&&)) (not <$> ((pure even) <*> (intro $ pure 1))) ((pure odd) <*> pro)

Line 1: Redundant bracket
Found:
(liftA2 (&&)) (not <$> ((pure even) <*> (intro $ pure 1)))
Why not:
liftA2 (&&) (not <$> ((pure even) <*> (intro $ pure 1)))Line 1: Redundant bracket
Found:
(pure even) <*> (intro $ pure 1)
Why not:
pure even <*> (intro $ pure 1)Line 1: Move brackets to avoid $
Found:
(pure even) <*> (intro $ pure 1)
Why not:
(pure even) <*> intro (pure 1)Line 1: Redundant bracket
Found:
(pure odd) <*> pro
Why not:
pure odd <*> pro

(True,[])

In [12]:
anEvenIntD = join $ (liftS <$> intro <$> pure <$> anEvenInt)

Line 1: Use =<<
Found:
join $ (liftS <$> intro <$> pure <$> anEvenInt)
Why not:
(liftS <$> intro <$> pure) =<< anEvenIntLine 1: Redundant $
Found:
$
Why not:
Line 1: Functor law
Found:
liftS <$> intro <$> pure
Why not:
(liftS . intro <$> pure)Line 1: Functor law
Found:
liftS <$> intro <$> pure <$> anEvenInt
Why not:
(liftS <$> intro) . pure <$> anEvenInt

# Negation in dynamic semantics

If we just fmap negation into the contained value, then "no even integer less than 6" should mean the same thing as "an even integer not less than 6".

In [13]:
($ []) $ runSS $ (liftM not) $ (liftM (< 6)) anEvenIntD

Line 1: Redundant bracket
Found:
(liftM not) $ (liftM (< 6)) anEvenIntD
Why not:
liftM not $ (liftM (< 6)) anEvenIntDLine 1: Use fmap
Found:
liftM
Why not:
fmapLine 1: Redundant bracket
Found:
(liftM (< 6)) anEvenIntD
Why not:
liftM (< 6) anEvenIntDLine 1: Use fmap
Found:
liftM
Why not:
fmap

[(False,[2]),(False,[4]),(True,[6]),(True,[8]),(True,[10])]

This isn't bad per se, in the sense that we would like to allow indefinites to scope above negation.

We simply evaluate its scope via dynamic closure.

In [14]:
($ []) $ runS $ clo $ (liftM (< 6)) anEvenIntD

Line 1: Redundant bracket
Found:
(liftM (< 6)) anEvenIntD
Why not:
liftM (< 6) anEvenIntDLine 1: Use fmap
Found:
liftM
Why not:
fmap

(True,[])

Can we play with the definition of dynamic closure to get the right results?

The following entry for dynamic closure throws out the false-tagged assignments completely; `clo2` thereby marks the scope of the indefinite.

In [15]:
clo2 :: SS Bool -> SS Bool
clo2 m = SS $ \g -> [(True,g')|(True,g') <- runSS m g]

In [16]:
 ($ []) $ runSS $ (liftM2 (&&)) ((liftM not) $ clo2 $ (liftM (< 6)) anEvenIntD) ((pure even) <*> (pure 2))

Line 1: Redundant bracket
Found:
(liftM2 (&&)) ((liftM not) $ clo2 $ (liftM (< 6)) anEvenIntD)
Why not:
liftM2 (&&) ((liftM not) $ clo2 $ (liftM (< 6)) anEvenIntD)Line 1: Redundant bracket
Found:
(liftM not) $ clo2 $ (liftM (< 6)) anEvenIntD
Why not:
liftM not $ clo2 $ (liftM (< 6)) anEvenIntDLine 1: Use fmap
Found:
liftM
Why not:
fmapLine 1: Redundant bracket
Found:
(liftM (< 6)) anEvenIntD
Why not:
liftM (< 6) anEvenIntDLine 1: Use fmap
Found:
liftM
Why not:
fmapLine 1: Redundant bracket
Found:
(pure even) <*> (pure 2)
Why not:
pure even <*> (pure 2)Line 1: Redundant bracket
Found:
(pure even) <*> (pure 2)
Why not:
(pure even) <*> pure 2

[(False,[2]),(False,[4])]

We can build in the uniqueness requirement as a kind of maximality check.

TODO: this still needs to be fixed.

In [21]:
maximalG :: [G] -> G
maximalG assignments = [maximum [g !! 0|g <- assignments ]]

Line 2: Use head
Found:
g !! 0
Why not:
head g